In [ ]:

import pandas as pd
import pyarrow.parquet as pq
import os


## prendo i primi due dataset per ogni anno

In [ ]:
base_path = "dati"
years = [2019, 2020, 2021, 2022]

tratte_disp_list = []
idsap_rischio_list = []

for year in years:
    year_path = os.path.join(base_path, str(year))

    tratte_file = os.path.join(year_path, f"tratte_disp_{year}.parquet")
    if os.path.exists(tratte_file):
        df_tratte = pd.read_parquet(tratte_file)
        tratte_disp_list.append(df_tratte)

    rischio_folder = os.path.join(year_path, f"idsap_rischio_{year}")
    if os.path.exists(rischio_folder):
        for file in os.listdir(rischio_folder):
            if file.endswith(".snappy.parquet") and file.startswith("part-"):
                file_path = os.path.join(rischio_folder, file)
                df_rischio = pd.read_parquet(file_path)
                idsap_rischio_list.append(df_rischio)

# Concatenazione dei dataset
disp_df = pd.concat(tratte_disp_list, ignore_index=True)
risk_df = pd.concat(idsap_rischio_list, ignore_index=True)

# Output finali
print("Tratte Disp Dataset Shape:", disp_df.shape)
print("ID SAP Rischio Dataset Shape:", risk_df.shape)

## Li sistemo secondo il codice gia mandato da Lorenza, metto i nan = 0 , aggiungo la colonna anno per dopo

In [ ]:
disp_df.columns = disp_df.columns.str.lower()
disp_df['odl'] = disp_df['odl'].astype(str)
disp_df['idsap'] = disp_df['idsap'].astype(str)
risk_df['idsap'] = risk_df['idsap'].astype(str)
disp_df['data'] = pd.to_datetime(disp_df['data'], errors='coerce')
risk_df['data'] = pd.to_datetime(risk_df['data'], errors='coerce')
print("Numero di ID SAP univoci in disp_df:", disp_df['idsap'].nunique())
print("Numero di ID SAP univoci in risk_df:", risk_df['idsap'].nunique())

disp_df['anno'] = disp_df['data'].dt.year
disp_df['mese'] = disp_df['data'].dt.month

disp_df['data'] = pd.to_datetime({'year': disp_df['anno'], 'month': disp_df['mese'], 'day': 1})
disp_df = disp_df.drop(columns=['anno', 'mese'])

disp_tot_df = disp_df.groupby(['idsap', 'data'])['odl'].count().reset_index()
disp_tot_df.rename(columns={'odl': 'disp_mensili'}, inplace=True)
disp_tot_df

merged_df = pd.merge(risk_df, disp_tot_df, on=['idsap', 'data'], how='left')
merged_df['disp_mensili'] = merged_df['disp_mensili'].fillna(0)
merged_df['date'] = merged_df['data'].dt.year.astype('int64')
merged_df

## carica i geometry e aggiungo l anno per mergarli dopo

In [ ]:
input_dir = "dati_finali_disp"

# Nomi dei file parquet
file_names = ["df1.parquet", "df2.parquet", "df3.parquet", "df4.parquet"]

# Caricare i file parquet e inserirli in una lista
dataframes = []
for file_name in file_names:
    file_path = os.path.join(input_dir, file_name)
    if os.path.exists(file_path):
        df = pd.read_parquet(file_path)
        dataframes.append(df)
        print(f"Loaded {file_name} with shape {df.shape}")
    else:
        print(f"File {file_name} not found!")

print(f"Total DataFrames loaded: {len(dataframes)}")
df1=dataframes[0]
df2=dataframes[1]
df3=dataframes[2]
df4=dataframes[3]

df1['date'] = 2019
df2['date'] = 2020
df3['date'] = 2021
df4['date'] = 2022

## UNISCO I DF GEOMETRY ASSIEME

In [ ]:
def concatenate_dataframes(*dataframes):
    """
    Concatena i DataFrame forniti in ordine, mettendo le righe di ciascun DataFrame
    dopo quelle del precedente.
    
    Args:
    - *dataframes (pd.DataFrame): Sequenza di DataFrame da concatenare.
    
    Returns:
    - pd.DataFrame: DataFrame risultante dalla concatenazione.
    """
    result = pd.concat(dataframes, ignore_index=True)
    return result

# Esempio di utilizzo:
merged_df_geometry = concatenate_dataframes(df1, df2, df3, df4)

## ORA ABBIAMO TUTTO
**merged_df**   
**merged_df_geometry le geometry per anno per ogni idsap**

In [ ]:
merged_df.rename(columns={"idsap": "IDSAP"}, inplace=True)


def colonna_unica(df,col_name):
    unique_elements = df[col_name].value_counts()
    print("LEN(DF):", len(df))
    print("LEN(unique,idsap):", len(unique_elements))
    if((len(unique_elements))<100):
        # Stampare i risultati
        print(f"Elementi unici nella colonna {col_name}:")
        for elem, count in unique_elements.items():
            print(f": {elem}, Count: {count}")

idsap_unici(merged_df,"IDSAP")
idsap_unici(merged_df_geometry,"IDSAP")

idsap_unici(df1,"IDSAP")
idsap_unici(df2,"IDSAP")
idsap_unici(df3,"IDSAP")
idsap_unici(df4,"IDSAP")


## PRENDI L INTERSEZIONE TRA DF_MERGED E GLI ALTRI

In [ ]:
def filter_by_intersection(df1, df2, column_name="IDSAP"):
    """
    Confronta gli elementi di una colonna tra due DataFrame e lascia solo l'intersezione.
    
    Args:
    - df1 (pd.DataFrame): Primo DataFrame.
    - df2 (pd.DataFrame): Secondo DataFrame.
    - column_name (str): Nome della colonna su cui basare l'intersezione.
    
    Returns:
    - (pd.DataFrame, pd.DataFrame): I due DataFrame filtrati.
    """
    print(f"Length of DataFrame 1 before filtering: {len(df1)}")
    print(f"Length of DataFrame 2 before filtering: {len(df2)}")
    
    # Trova l'intersezione degli elementi nella colonna specificata
    common_elements = set(df1[column_name]).intersection(set(df2[column_name]))
    
    # Filtra entrambi i DataFrame mantenendo solo i valori comuni
    df1_filtered = df1[df1[column_name].isin(common_elements)].reset_index(drop=True)
    df2_filtered = df2[df2[column_name].isin(common_elements)].reset_index(drop=True)
    
    print(f"Length of DataFrame 1 after filtering: {len(df1_filtered)}")
    print(f"Length of DataFrame 2 after filtering: {len(df2_filtered)}")
    
    return df1_filtered, df2_filtered


merged_df_filtered,merged_df_geometry_filtered = filter_by_intersection(merged_df, merged_df_geometry, column_name="IDSAP")


In [ ]:
merged_df_filtered.head(5)


In [ ]:
merged_df_geometry_filtered.head(5)

## CREO DATASET FINALE GIGANTE CON TUTTO MERGANDO LE GEOMETRY E LE TRATTE

In [ ]:

def merge_on_idsap_and_date(df1, df2):
    """
    Esegue un merge tra due DataFrame sulla base delle colonne 'IDSAP' e 'date'.
    
    Args:
    - df1 (pd.DataFrame): Il primo DataFrame.
    - df2 (pd.DataFrame): Il secondo DataFrame.
    
    Returns:
    - pd.DataFrame: Il DataFrame risultante dal merge.
    """
    merged_df = pd.merge(df1, df2, on=['IDSAP', 'date'], how='inner')
    return merged_df

# Esempio di utilizzo
final_df = merge_on_idsap_and_date(merged_df_filtered, merged_df_geometry_filtered)

# Stampa i risultati
print(f"Shape of df1: {merged_df_filtered.shape}")
print(f"Shape of df2: {merged_df_geometry_filtered.shape}")
print(f"Shape of merged_df: {final_df.shape}")

In [ ]:
final_df_sorted_idsap_date = final_df.sort_values(by=['IDSAP', 'date'])
final_df_sorted_idsap_date.head(5)



## Save dataset, un po inutile ci mette piu a fare l upload che a crearlo :)

In [ ]:
output_path = "final_df_sorted_idsap_date.parquet"

# Save the DataFrame as a Parquet file
final_df_sorted_idsap_date.to_parquet(output_path, index=False)

print(f"DataFrame successfully saved to {output_path}")

In [ ]:
# Specifica il percorso del file Parquet
input_path = "final_df_sorted_idsap_date.parquet"

# Carica il DataFrame da Parquet
final_df_sorted_idsap_date = pd.read_parquet(input_path)

In [ ]:
def colonna_unica(df,col_name):
    unique_elements = df[col_name].value_counts()
    print("LEN(DF):", len(df))
    print("LEN(unique,idsap):", len(unique_elements))
    if((len(unique_elements))<100):
        # Stampare i risultati
        print(f"Elementi unici nella colonna {col_name}:")
        for elem, count in unique_elements.items():
            print(f": {elem}, Count: {count}")

colonna_unica(final_df_sorted_idsap_date,"CODSISTEMA")

## AGGIUNGO ANNO E MESE AL DATASET FINALE PER POTER GRUPPARE DOPO PER IMPIANTO

In [ ]:
"""
Index(['data', 'IDSAP', 'risk_level', 'disp_mensili', 'date', 'TIPO',
    'MATERIALE', 'DIAMETRO', 'ANNO_POSA', 'CODSISTEMA', 'lenght',
    'is_closed', 'is_simple', 'is_ring', 'minimum_clearance', 'centroid_x',
    'centroid_y', 'bound_0', 'bound_1', 'bound_2', 'bound_3',
    'x_coordinates', 'y_coordinates'],
    dtype='object')
"""
final_df_sorted_idsap_date['year'] = final_df_sorted_idsap_date['data'].dt.year
final_df_sorted_idsap_date['month'] = final_df_sorted_idsap_date['data'].dt.month

final_df_sorted_idsap_date.columns

## FINAL_DF HA QUELLI PRESENTI 48 VOLTE , DF_GROUPED ANCHE QUELLI CON QUALCHE NAN

In [ ]:
# Seleziona solo le colonne numeriche
numeric_columns = final_df_sorted_idsap_date.select_dtypes(include=['float64', 'int64']).columns

# Esegui il groupby per 'CODSISTEMA', 'year', 'month' e calcola la media per tutte le colonne numeriche
df_grouped = final_df_sorted_idsap_date.groupby(['CODSISTEMA', 'year', 'month'], as_index=False)[numeric_columns].mean()

counts_per_codsistema = df_grouped.groupby('CODSISTEMA').size()
# Filtra i CODSISTEMA che hanno esattamente 48 righe
valid_codsistema = counts_per_codsistema[counts_per_codsistema == 48].index

# Filtra il DataFrame finale per mantenere solo i CODSISTEMA con 48 righe
final_df_filtered = df_grouped[df_grouped['CODSISTEMA'].isin(valid_codsistema)]

# Verifica la dimensione del nuovo DataFrame

print(final_df_filtered.shape)
print(df_grouped.shape)



In [ ]:
final_df_filtered